# 10_PhysicsVerification - Physics-SR Framework v4.1

## Stage 3.2: Dimensional Consistency + Physical Bounds Check

**Author:** Zhengze Zhang  
**Affiliation:** Department of Statistics, Columbia University  
**Contact:** zz3239@columbia.edu  
**Date:** January 2026  
**Version:** 4.1 (Structure-Guided Feature Library Enhancement + Computational Optimization)

---

### Purpose

Verify discovered equations satisfy physics constraints:
1. **Dimensional Consistency:** All terms have same dimensions as target
2. **Physical Bounds:** Predictions satisfy physical constraints (e.g., non-negativity)

This is a **NEW implementation** module for v4.1.

### v4.1 Features

| Feature | Description |
|---------|-------------|
| Numerical dimensional check | Tolerance-based comparison for non-integer exponents |
| Physics score | Overall score (0-1) for equation validity |
| Enhanced parsing | Support for source-tagged library terms |
| Comprehensive reporting | v4.1 format report with all details |

### Dimensional Analysis

For each term in the equation, compute dimensional exponents:
$$\dim(\text{term}) = \sum_j \text{exponent}_j \times \dim(\text{variable}_j)$$

### Tolerance for Non-Integer Exponents

PySR may find exponents like 2.47 when the true value is 2.5. We use:
- `dim_tolerance = 0.05` for dimensional comparison
- This accepts [0, -3.02, 0, 0] $\approx$ [0, -3, 0, 0]

### Reference

- Framework v4.0/v4.1 Section 5.2: Physics Verification

---
## Section 1: Header and Imports

In [ ]:
"""
10_PhysicsVerification.ipynb - Dimensional Consistency + Physical Bounds
=========================================================================

Three-Stage Physics-Informed Symbolic Regression Framework v4.1

This module provides:
- PhysicsVerifier: Complete physics verification with:
  - Dimensional consistency check (with tolerance)
  - Physical bounds violation detection
  - Physics score computation (0-1)
  - Equation parsing for dimensional analysis

NEW in v4.1:
- Full implementation of verify() method
- check_dimensional_consistency() method
- check_physical_bounds() method
- parse_equation_for_verification() method
- physics_score in output

Author: Zhengze Zhang
Affiliation: Department of Statistics, Columbia University
Contact: zz3239@columbia.edu
"""

# Import core module
%run 00_Core.ipynb

In [ ]:
# Additional imports for Physics Verification
import re
import sympy as sp
from typing import Dict, List, Tuple, Optional, Any, Union

print("10_PhysicsVerification v4.1: Additional imports successful.")

---
## Section 2: Class Definition

In [ ]:
# ==============================================================================
# PHYSICS VERIFIER CLASS (v4.1 NEW)
# ==============================================================================

class PhysicsVerifier:
    """
    Physics Verification Module (v4.1 NEW Implementation).
    
    Verifies discovered equations satisfy physics constraints:
    1. Dimensional consistency - all terms have same dimensions
    2. Physical bounds - predictions within physical constraints
    
    v4.1 Features:
    - Numerical dimensional check with tolerance
    - Physics score computation (0-1)
    - Support for source-tagged library terms
    - Enhanced equation parsing
    
    Attributes
    ----------
    dim_tolerance : float
        Tolerance for dimensional comparison (default: 0.05)
    bounds_tolerance : float
        Allowed fraction of bound violations (default: 0.01)
    
    Methods
    -------
    verify(equation_terms, variable_dims, target_dims, X_val, y_pred, physical_bounds) -> Dict
        Perform complete physics verification
    check_dimensional_consistency(equation_terms, variable_dims, target_dims) -> Dict
        Check dimensional consistency only
    check_physical_bounds(y_pred, physical_bounds, target_name) -> Dict
        Check physical bounds only
    parse_equation_for_verification(equation_str, feature_names) -> List[Dict]
        Parse equation string for dimensional analysis
    
    Reference
    ---------
    Framework v4.0/v4.1 Section 5.2: Physics Verification
    
    Examples
    --------
    >>> verifier = PhysicsVerifier(dim_tolerance=0.05)
    >>> result = verifier.verify(terms, var_dims, target_dims, X, y_pred, bounds)
    >>> print(f"Physics score: {result['physics_score']:.2f}")
    """
    
    def __init__(
        self,
        dim_tolerance: float = DEFAULT_DIM_TOLERANCE,
        bounds_tolerance: float = 0.01
    ):
        """
        Initialize PhysicsVerifier.
        
        Parameters
        ----------
        dim_tolerance : float
            Tolerance for dimensional exponent comparison.
            Default: 0.05 (accepts 2.47 ~ 2.5)
        bounds_tolerance : float
            Fraction of violations allowed before flagging.
            Default: 0.01 (1% violations OK)
        """
        self.dim_tolerance = dim_tolerance
        self.bounds_tolerance = bounds_tolerance
        
        # Internal state
        self._dim_result = None
        self._bounds_result = None
        self._physics_score = None
        self._verification_complete = False
    
    def verify(
        self,
        equation_terms: List[Dict],
        variable_dims: Dict[str, List[float]],
        target_dims: List[float],
        X_val: np.ndarray = None,
        y_pred: np.ndarray = None,
        physical_bounds: Dict = None
    ) -> Dict[str, Any]:
        """
        Perform complete physics verification.
        
        Parameters
        ----------
        equation_terms : List[Dict]
            List of term dictionaries, each with:
            - 'variables': Dict[str, float] mapping var name to exponent
            - 'coefficient': float (optional)
            - 'name': str (optional)
        variable_dims : Dict[str, List[float]]
            Variable dimensions {var_name: [M, L, T, Theta]}
        target_dims : List[float]
            Target dimensions [M, L, T, Theta]
        X_val : np.ndarray, optional
            Validation feature matrix (for future extensions)
        y_pred : np.ndarray, optional
            Predicted values for bounds check
        physical_bounds : Dict, optional
            Physical bounds {target_name: {min, max}} or {min, max}
            
        Returns
        -------
        Dict
            - dim_consistent: Boolean
            - dim_details: Detailed dimension analysis
            - matches_target: Boolean
            - bounds_violations: Violation statistics
            - physics_score: Overall score (0-1)
            - overall_valid: Boolean (all checks pass)
        """
        # Dimensional consistency check
        self._dim_result = self.check_dimensional_consistency(
            equation_terms, variable_dims, target_dims
        )
        
        # Physical bounds check
        if y_pred is not None and physical_bounds is not None:
            self._bounds_result = self.check_physical_bounds(
                y_pred, physical_bounds
            )
        else:
            self._bounds_result = {
                'passed': True,
                'n_violations': 0,
                'violation_rate': 0.0,
                'n_negative': 0,
                'n_exceed_max': 0,
                'n_total': 0
            }
        
        # Compute physics score
        self._physics_score = self._compute_physics_score()
        
        # Overall validity
        overall_valid = (
            self._dim_result['consistent'] and
            self._dim_result['matches_target'] and
            self._bounds_result['passed']
        )
        
        self._verification_complete = True
        
        return {
            'dim_consistent': self._dim_result['consistent'],
            'matches_target': self._dim_result['matches_target'],
            'term_dimensions': self._dim_result['term_dimensions'],
            'dim_details': self._dim_result['details'],
            'bounds_passed': self._bounds_result['passed'],
            'bounds_violations': {
                'n_violations': self._bounds_result['n_violations'],
                'violation_rate': self._bounds_result['violation_rate'],
                'n_negative': self._bounds_result['n_negative'],
                'n_exceed_max': self._bounds_result['n_exceed_max']
            },
            'physics_score': self._physics_score,
            'overall_valid': overall_valid
        }
    
    def check_dimensional_consistency(
        self,
        equation_terms: List[Dict],
        variable_dims: Dict[str, List[float]],
        target_dims: List[float]
    ) -> Dict[str, Any]:
        """
        Check dimensional consistency numerically.
        
        Uses tolerance-based comparison to handle non-integer exponents.
        
        Parameters
        ----------
        equation_terms : List[Dict]
            Term exponents, each dict has 'variables' mapping var to exponent
        variable_dims : Dict[str, List[float]]
            Variable dimensions {var_name: [M, L, T, Theta]}
        target_dims : List[float]
            Target dimensions [M, L, T, Theta]
            
        Returns
        -------
        Dict
            - consistent: Boolean (all terms have same dimensions)
            - matches_target: Boolean (dimensions match target)
            - term_dimensions: List of computed dimensions per term
            - details: Detailed analysis per term
        """
        if len(equation_terms) == 0:
            return {
                'consistent': True,
                'matches_target': True,
                'term_dimensions': [],
                'details': []
            }
        
        target_dims = np.array(target_dims, dtype=np.float64)
        n_dims = len(target_dims)
        
        term_dimensions = []
        details = []
        
        for i, term in enumerate(equation_terms):
            term_dim = self._compute_term_dimensions(term, variable_dims, n_dims)
            term_dimensions.append(term_dim)
            
            matches = np.allclose(term_dim, target_dims, atol=self.dim_tolerance)
            
            details.append({
                'term_index': i,
                'name': term.get('name', f'term_{i}'),
                'dimensions': term_dim.tolist(),
                'target': target_dims.tolist(),
                'matches_target': matches,
                'max_deviation': float(np.max(np.abs(term_dim - target_dims)))
            })
        
        term_dimensions = np.array(term_dimensions)
        
        # Check if all terms are consistent with each other
        if len(term_dimensions) > 1:
            all_consistent = all(
                np.allclose(term_dimensions[0], td, atol=self.dim_tolerance)
                for td in term_dimensions[1:]
            )
        else:
            all_consistent = True
        
        # Check if terms match target
        matches_target = len(term_dimensions) > 0 and np.allclose(
            term_dimensions[0], target_dims, atol=self.dim_tolerance
        )
        
        return {
            'consistent': all_consistent,
            'matches_target': matches_target,
            'term_dimensions': term_dimensions.tolist() if len(term_dimensions) > 0 else [],
            'details': details
        }
    
    def _compute_term_dimensions(
        self,
        term: Dict[str, float],
        variable_dims: Dict[str, List[float]],
        n_dims: int = 4
    ) -> np.ndarray:
        """
        Compute dimensions for a single term.
        
        dim(term) = sum_j exponent_j * dim(variable_j)
        
        Parameters
        ----------
        term : Dict
            Term structure. Can be:
            - {'variables': {'var1': exp1, ...}, ...} (standard format)
            - {'var1': exp1, 'var2': exp2, ...} (compact format)
        variable_dims : Dict[str, List[float]]
            Variable dimensions
        n_dims : int
            Number of dimensions (default: 4 for M, L, T, Theta)
            
        Returns
        -------
        np.ndarray
            Computed dimensions
        """
        dims = np.zeros(n_dims, dtype=np.float64)
        
        # Handle both formats
        if 'variables' in term:
            variables = term['variables']
        else:
            # Compact format: term is {var_name: exponent}
            variables = {k: v for k, v in term.items() 
                        if k not in ['coefficient', 'name', 'source']}
        
        for var_name, exponent in variables.items():
            if var_name in variable_dims:
                var_dim = np.array(variable_dims[var_name], dtype=np.float64)
                dims += exponent * var_dim
            # If variable not in var_dims, assume dimensionless (contributes 0)
        
        return dims
    
    def check_physical_bounds(
        self,
        y_pred: np.ndarray,
        physical_bounds: Dict,
        target_name: str = 'target'
    ) -> Dict[str, Any]:
        """
        Check physical bounds violations.
        
        Parameters
        ----------
        y_pred : np.ndarray
            Predicted values
        physical_bounds : Dict
            Physical bounds. Can be:
            - {'min': val, 'max': val} (simple format)
            - {target_name: {'min': val, 'max': val}} (nested format)
        target_name : str
            Name of target variable in nested bounds dict
            
        Returns
        -------
        Dict
            - passed: Boolean (violation_rate < tolerance)
            - n_violations: Total violations
            - violation_rate: Fraction of violations
            - n_negative: Negative value violations (if min >= 0)
            - n_exceed_max: Maximum exceeded violations
        """
        n_total = len(y_pred)
        
        if n_total == 0:
            return {
                'passed': True,
                'n_violations': 0,
                'violation_rate': 0.0,
                'n_negative': 0,
                'n_exceed_max': 0,
                'n_total': 0
            }
        
        # Handle nested format
        if target_name in physical_bounds:
            bounds = physical_bounds[target_name]
        else:
            bounds = physical_bounds
        
        n_negative = 0
        n_exceed_max = 0
        
        if 'min' in bounds:
            n_negative = int(np.sum(y_pred < bounds['min']))
        
        if 'max' in bounds:
            n_exceed_max = int(np.sum(y_pred > bounds['max']))
        
        n_violations = n_negative + n_exceed_max
        violation_rate = n_violations / n_total
        
        passed = violation_rate <= self.bounds_tolerance
        
        return {
            'passed': passed,
            'n_violations': n_violations,
            'violation_rate': violation_rate,
            'n_negative': n_negative,
            'n_exceed_max': n_exceed_max,
            'n_total': n_total
        }
    
    def _compute_physics_score(
        self
    ) -> float:
        """
        Compute overall physics score (0-1).
        
        Score components:
        - 0.5 for dimensional consistency
        - 0.5 for bounds satisfaction
        
        Returns
        -------
        float
            Physics score in [0, 1]
        """
        score = 0.0
        
        # Dimensional component (0.5)
        if self._dim_result is not None:
            if self._dim_result['consistent'] and self._dim_result['matches_target']:
                score += 0.5
            elif self._dim_result['consistent'] or self._dim_result['matches_target']:
                score += 0.25
        
        # Bounds component (0.5)
        if self._bounds_result is not None:
            if self._bounds_result['passed']:
                score += 0.5
            else:
                # Partial credit based on violation rate
                vr = self._bounds_result['violation_rate']
                score += 0.5 * max(0, 1 - vr * 10)  # Linear penalty
        
        return score
    
    def parse_equation_for_verification(
        self,
        equation_str: str,
        feature_names: List[str]
    ) -> List[Dict]:
        """
        Parse equation string to extract term exponents for dimensional check.
        
        Handles source-tagged terms like '[PySR] x**2 * y**3'.
        
        Parameters
        ----------
        equation_str : str
            Equation string, e.g., "0.89*q_c**2.47*N_d**(-1.79)"
        feature_names : List[str]
            List of feature names (base names without tags)
            
        Returns
        -------
        List[Dict]
            List of term structures for dimensional check
        """
        terms = []
        
        # Remove source tags
        clean_str = re.sub(r'\[\w+\]\s*', '', equation_str)
        
        # Split by + (handling +/- signs)
        clean_str = clean_str.replace(' ', '')
        clean_str = clean_str.replace('-', '+-')
        term_strs = [t for t in clean_str.split('+') if t]
        
        for term_str in term_strs:
            term = self._parse_single_term(term_str, feature_names)
            if term:
                terms.append(term)
        
        return terms
    
    def _parse_single_term(
        self,
        term_str: str,
        feature_names: List[str]
    ) -> Dict:
        """
        Parse a single term string.
        
        Parameters
        ----------
        term_str : str
            Single term string
        feature_names : List[str]
            Feature names
            
        Returns
        -------
        Dict
            Term structure with 'variables', 'coefficient', 'name'
        """
        term = {
            'coefficient': 1.0,
            'variables': {},
            'name': term_str
        }
        
        # Extract coefficient (leading number)
        coef_match = re.match(r'^([+-]?\d*\.?\d+)\*?', term_str)
        if coef_match:
            try:
                term['coefficient'] = float(coef_match.group(1))
            except ValueError:
                pass
        
        # Extract variable exponents
        for var_name in feature_names:
            # Pattern: var_name**exponent or var_name**(-exponent) or var_name^exponent
            patterns = [
                rf'{re.escape(var_name)}\*\*\(([+-]?\d*\.?\d+)\)',  # x**(2.47)
                rf'{re.escape(var_name)}\*\*([+-]?\d*\.?\d+)',       # x**2.47
                rf'{re.escape(var_name)}\^\(([+-]?\d*\.?\d+)\)',    # x^(2.47)
                rf'{re.escape(var_name)}\^([+-]?\d*\.?\d+)',         # x^2.47
            ]
            
            found = False
            for pattern in patterns:
                match = re.search(pattern, term_str)
                if match:
                    try:
                        term['variables'][var_name] = float(match.group(1))
                        found = True
                        break
                    except ValueError:
                        pass
            
            if not found and var_name in term_str:
                # Variable without explicit exponent = exponent 1
                term['variables'][var_name] = 1.0
        
        return term
    
    def print_verification_report(self) -> None:
        """
        Print detailed verification report in v4.1 format.
        """
        if not self._verification_complete:
            print("Verification not yet performed. Call verify() first.")
            return
        
        print("=" * 70)
        print("=== Physics Verification (v4.1) ===")
        print("=" * 70)
        print()
        
        # Dimensional Consistency
        dim_status = "PASS" if self._dim_result['consistent'] and self._dim_result['matches_target'] else "FAIL"
        print(f"Dimensional Consistency: {dim_status}")
        
        if self._dim_result['details']:
            for detail in self._dim_result['details']:
                target_str = f"{detail['target']}"
                dims_str = f"{detail['dimensions']}"
                status = "matches" if detail['matches_target'] else "MISMATCH"
                print(f"  {detail['name']}: {dims_str} {status} target {target_str}")
        print()
        
        # Physical Bounds
        bounds_status = "PASS" if self._bounds_result['passed'] else "FAIL"
        print(f"Physical Bounds Check: {bounds_status}")
        print(f"  Negative violations: {self._bounds_result['n_negative']} ({self._bounds_result['violation_rate']*100:.1f}%)")
        print(f"  Max exceeded: {self._bounds_result['n_exceed_max']} ({self._bounds_result['violation_rate']*100:.1f}%)")
        print()
        
        # Overall Physics Score
        print(f"Overall Physics Score: {self._physics_score:.2f}")
        print()
        print("=" * 70)

print("PhysicsVerifier class v4.1 defined.")

---
## Section 3: Internal Tests

In [ ]:
# ==============================================================================
# TEST CONTROL FLAG
# ==============================================================================

_RUN_TESTS = False  # Set to True to run internal tests

if _RUN_TESTS:
    print("=" * 70)
    print(" RUNNING INTERNAL TESTS FOR 10_PhysicsVerification v4.1")
    print("=" * 70)

In [ ]:
# ==============================================================================
# TEST 1: Dimensional Consistency - Correct Equation
# ==============================================================================

if _RUN_TESTS:
    print()
    print_section_header("Test 1: Dimensional Consistency - Correct Equation")
    
    # y = x^2 where x is length, y is area
    variable_dims = {
        'x': [0, 1, 0, 0]  # length [M, L, T, Theta]
    }
    target_dims = [0, 2, 0, 0]  # area
    
    terms = [{
        'variables': {'x': 2.0},
        'coefficient': 1.0,
        'name': 'x^2'
    }]
    
    verifier = PhysicsVerifier(dim_tolerance=0.05)
    result = verifier.verify(terms, variable_dims, target_dims)
    
    print(f"Equation: y = x^2")
    print(f"x dimensions: [0, 1, 0, 0] (length)")
    print(f"Target: [0, 2, 0, 0] (area)")
    print(f"Computed: {result['term_dimensions'][0]}")
    print(f"Matches target: {result['matches_target']}")
    print(f"Physics score: {result['physics_score']:.2f}")
    
    if result['matches_target']:
        print("[PASS] Dimensional consistency verified")
    else:
        print("[FAIL] Unexpected dimension mismatch")

In [ ]:
# ==============================================================================
# TEST 2: Numerical Dimension Check with Tolerance
# ==============================================================================

if _RUN_TESTS:
    print()
    print_section_header("Test 2: Numerical Dimension Check with Tolerance")
    
    variable_dims = {
        'x': [0, 1, 0, 0]  # length
    }
    target_dims = [0, 2.5, 0, 0]  # target dimension
    
    # PySR might find 2.47 instead of 2.5
    terms = [{
        'variables': {'x': 2.47},
        'coefficient': 1.0,
        'name': 'x^2.47'
    }]
    
    tolerances = [0.01, 0.05, 0.1]
    
    print(f"Target: [0, 2.5, 0, 0]")
    print(f"Computed: [0, 2.47, 0, 0]")
    print(f"Difference: 0.03")
    print()
    print(f"{'Tolerance':<12} {'Matches':<10} {'Score'}")
    print("-" * 35)
    
    for tol in tolerances:
        verifier = PhysicsVerifier(dim_tolerance=tol)
        result = verifier.verify(terms, variable_dims, target_dims)
        print(f"{tol:<12.2f} {str(result['matches_target']):<10} {result['physics_score']:.2f}")
    
    print()
    print("Note: dim_tolerance=0.05 is recommended for SR applications")

In [ ]:
# ==============================================================================
# TEST 3: Physical Bounds Violations
# ==============================================================================

if _RUN_TESTS:
    print()
    print_section_header("Test 3: Physical Bounds Violations")
    
    # Predictions with some violations
    np.random.seed(42)
    y_pred = np.concatenate([
        np.random.uniform(0, 10, 95),   # 95 valid
        np.array([-1, -2, -0.5]),        # 3 negative (invalid)
        np.array([100, 200])             # 2 too large
    ])
    
    bounds = {'min': 0, 'max': 50}
    
    verifier = PhysicsVerifier(bounds_tolerance=0.05)  # Allow 5% violations
    result = verifier.verify([], {}, [], y_pred=y_pred, physical_bounds=bounds)
    
    print(f"Total predictions: 100")
    print(f"Negative values: 3")
    print(f"Values > max: 2")
    print(f"Violation rate: {result['bounds_violations']['violation_rate']:.2%}")
    print(f"Bounds passed (5% tolerance): {result['bounds_passed']}")
    print(f"Physics score: {result['physics_score']:.2f}")
    
    # 5% of 100 = 5 violations allowed, we have 5 exactly
    if result['bounds_passed']:
        print("[PASS] Bounds check working correctly")
    else:
        print("[INFO] Bounds exceeded threshold")

In [ ]:
# ==============================================================================
# TEST 4: Equation Parsing for Verification
# ==============================================================================

if _RUN_TESTS:
    print()
    print_section_header("Test 4: Equation Parsing for Verification")
    
    equation = "0.89*q_c**2.47*N_d**(-1.79)"
    feature_names = ['q_c', 'N_d']
    
    verifier = PhysicsVerifier()
    terms = verifier.parse_equation_for_verification(equation, feature_names)
    
    print(f"Equation: {equation}")
    print(f"Parsed terms:")
    for i, term in enumerate(terms):
        print(f"  Term {i+1}:")
        print(f"    Coefficient: {term['coefficient']:.2f}")
        print(f"    Variables: {term['variables']}")
    
    # Verify parsing
    if len(terms) == 1:
        term = terms[0]
        if (abs(term['coefficient'] - 0.89) < 0.01 and
            abs(term['variables'].get('q_c', 0) - 2.47) < 0.01 and
            abs(term['variables'].get('N_d', 0) - (-1.79)) < 0.01):
            print("[PASS] Equation parsed correctly")
        else:
            print("[FAIL] Parsing error")
    else:
        print(f"[INFO] Parsed {len(terms)} terms")

In [ ]:
# ==============================================================================
# TEST 5: Full Verification Report
# ==============================================================================

if _RUN_TESTS:
    print()
    print_section_header("Test 5: Full Verification Report")
    
    # Warm rain equation example
    variable_dims = {
        'q_c': [0, 0, 0, 0],     # kg/kg (dimensionless mass ratio)
        'N_d': [0, -3, 0, 0]     # m^-3 (number concentration)
    }
    target_dims = [0, 0, -1, 0]  # s^-1 (rate)
    
    # For dimensional consistency, the coefficient must have dimensions
    # to make the equation balance. Here we test structure only.
    terms = [
        {'variables': {'q_c': 2.47, 'N_d': -1.79}, 'name': 'q_c^2.47 * N_d^-1.79'},
    ]
    
    # Generate predictions
    np.random.seed(42)
    y_pred = np.random.uniform(0, 1e-6, 100)  # Valid autoconversion rates
    bounds = {'min': 0, 'max': 1e-3}
    
    verifier = PhysicsVerifier(dim_tolerance=0.05)
    result = verifier.verify(terms, variable_dims, target_dims, y_pred=y_pred, physical_bounds=bounds)
    
    verifier.print_verification_report()
    
    print(f"\nFull result dictionary keys: {list(result.keys())}")

In [ ]:
# ==============================================================================
# TEST 6: Compact Term Format (v4.1)
# ==============================================================================

if _RUN_TESTS:
    print()
    print_section_header("Test 6: Compact Term Format (v4.1)")
    
    # Compact format: {'q_c': 2.47, 'N_d': -1.79} instead of {'variables': {...}}
    variable_dims = {
        'q_c': [0, 0, 0, 0],
        'N_d': [0, -3, 0, 0]
    }
    target_dims = [0, 5.37, 0, 0]  # Computed: 2.47*[0,0,0,0] + (-1.79)*[0,-3,0,0] = [0, 5.37, 0, 0]
    
    # Compact format terms
    compact_terms = [
        {'q_c': 2.47, 'N_d': -1.79}
    ]
    
    verifier = PhysicsVerifier(dim_tolerance=0.1)
    result = verifier.check_dimensional_consistency(compact_terms, variable_dims, target_dims)
    
    print(f"Compact term format: {compact_terms[0]}")
    print(f"Computed dimensions: {result['term_dimensions'][0]}")
    print(f"Target: {target_dims}")
    print(f"Matches: {result['matches_target']}")
    
    if result['matches_target']:
        print("[PASS] Compact format supported")
    else:
        print("[INFO] Check dimension computation")

---
## Section 4: Module Summary

In [ ]:
# ==============================================================================
# MODULE SUMMARY
# ==============================================================================

print("=" * 70)
print(" 10_PhysicsVerification.ipynb v4.1 - Module Summary")
print("=" * 70)
print()
print("CLASS: PhysicsVerifier (v4.1 NEW Implementation)")
print("-" * 70)
print()
print("Purpose:")
print("  Verify discovered equations satisfy physics constraints:")
print("  - Dimensional consistency across all terms")
print("  - Physical bounds on predictions")
print()
print("v4.1 Features:")
print("  - Numerical dimensional check with tolerance")
print("  - physics_score computation (0-1)")
print("  - Support for source-tagged library terms")
print("  - Compact and standard term formats")
print()
print("Main Methods:")
print("  verify(equation_terms, variable_dims, target_dims, ...) -> Dict")
print("      Full verification with dimensional and bounds checks")
print("      Returns: dim_consistent, matches_target, physics_score, ...")
print()
print("  check_dimensional_consistency(terms, var_dims, target_dims) -> Dict")
print("      Dimensional check only")
print()
print("  check_physical_bounds(y_pred, bounds, target_name) -> Dict")
print("      Bounds check only")
print()
print("  parse_equation_for_verification(equation_str, feature_names) -> List")
print("      Parse equation string for dimensional analysis")
print()
print("  print_verification_report()")
print("      Print detailed verification results")
print()
print("Expected Output Format (v4.1):")
print("-" * 70)
print("""
=== Physics Verification (v4.1) ===

Dimensional Consistency: PASS
  Term 1: [0, 0, -1, 0] matches target [0, 0, -1, 0]
  Term 2: [0, 0, -1, 0] matches target [0, 0, -1, 0]

Physical Bounds Check: PASS
  Negative violations: 0 (0.0%)
  Max exceeded: 0 (0.0%)

Overall Physics Score: 1.00
""")
print()
print("Usage Example:")
print("-" * 70)
print("""
# Define variable dimensions [M, L, T, Theta]
variable_dims = {
    'q_c': [0, 0, 0, 0],   # dimensionless
    'N_d': [0, -3, 0, 0]   # m^-3
}
target_dims = [0, 0, -1, 0]  # rate (s^-1)

# Physical bounds
bounds = {'min': 0, 'max': 1e-3}

# Verify
verifier = PhysicsVerifier(dim_tolerance=0.05)
result = verifier.verify(terms, variable_dims, target_dims, y_pred=y_pred, physical_bounds=bounds)

print(f"Physics score: {result['physics_score']:.2f}")
print(f"Overall valid: {result['overall_valid']}")
""")
print()
print("=" * 70)
print("Module loaded successfully. Import via: %run 10_PhysicsVerification.ipynb")
print("=" * 70)